<a href="https://colab.research.google.com/github/elizabethts/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/Unit_3_Sprint_2_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install psycopg2-binary

In [0]:
import psycopg2
import pandas

In [0]:
# elephantSQL details

dbname = 'kvobzcdf'
user = 'kvobzcdf'
password = 'Z6GcK2XttNxPBRVoCVBM-_FgYLdfDJst'
host = 'raja.db.elephantsql.com'


In [0]:
# create connection to postgres

pg_conn = psycopg2.connect(dbname=dbname, user=user,
                           password=password, host=host)

In [5]:
pg_conn

<connection object at 0x7fd0e696f2b8; dsn: 'user=kvobzcdf password=xxx dbname=kvobzcdf host=raja.db.elephantsql.com', closed: 0>

In [0]:
pg_curs = pg_conn.cursor()

In [7]:
pg_curs.execute('SELECT * FROM test_table;')
pg_curs.fetchall()

[(1, 'A row name', None),
 (2, 'Another row, with JSON', {'a': 1, 'b': ['dog', 'cat', 42], 'c': True})]

###Move RPG data from SQLite to PostgreSQL
####ETL - Extract, Transform, Load

In [8]:
!wget https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true

--2019-08-13 19:58:05--  https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2019-08-13 19:58:05--  https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2019-08-13 19:58:05--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module1-in

In [9]:
ls -alh

total 512K
drwxr-xr-x 1 root root 4.0K Aug 13 19:58  ./
drwxr-xr-x 1 root root 4.0K Aug 13 18:56  ../
drwxr-xr-x 1 root root 4.0K Aug  7 16:05  .config/
-rw-r--r-- 1 root root 248K Aug 13 19:54  rpg_db.sqlite3
-rw-r--r-- 1 root root 248K Aug 13 19:58 'rpg_db.sqlite3?raw=true'
drwxr-xr-x 1 root root 4.0K Aug  2 16:06  sample_data/


In [0]:
mv 'rpg_db.sqlite3?raw=true' rpg_db.sqlite3

In [0]:
import sqlite3

In [0]:
sl_conn = sqlite3.connect('rpg_db.sqlite3')
sl_curs = sl_conn.cursor()

In [13]:
sl_curs.execute('SELECT COUNT(*) FROM charactercreator_character;').fetchall()

[(302,)]

In [0]:
characters = sl_curs.execute('SELECT * FROM charactercreator_character;').fetchall()

In [15]:
characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [16]:
len(characters)

302

In [17]:
# extract table schema
sl_curs.execute('PRAGMA table_info(charactercreator_character);').fetchall()

[(0, 'character_id', 'integer', 1, None, 1),
 (1, 'name', 'varchar(30)', 1, None, 0),
 (2, 'level', 'integer', 1, None, 0),
 (3, 'exp', 'integer', 1, None, 0),
 (4, 'hp', 'integer', 1, None, 0),
 (5, 'strength', 'integer', 1, None, 0),
 (6, 'intelligence', 'integer', 1, None, 0),
 (7, 'dexterity', 'integer', 1, None, 0),
 (8, 'wisdom', 'integer', 1, None, 0)]

In [0]:
# transform

create_character_table = """
  CREATE TABLE charactercreator_character (
   character_id SERIAL PRIMARY KEY,
   name VARCHAR(30),
   level INT,
   exp INT,
   hp INT,
   strength INT,
   intelligence INT,
   dexterity INT,
   wisdom INT
  );
"""


In [0]:
pg_curs.execute(create_character_table)

In [0]:
show_tables = """
SELECT
   *
FROM
   pg_catalog.pg_tables
WHERE
   schemaname != 'pg_catalog'
AND schemaname != 'information_schema';
"""

In [22]:
pg_curs.execute(show_tables)
pg_curs.fetchall()

[('public', 'test_table', 'kvobzcdf', None, True, False, False, False),
 ('public',
  'charactercreator_character',
  'kvobzcdf',
  None,
  True,
  False,
  False,
  False)]

In [0]:
# Load data
for character in characters:
  insert_character = """
    INSERT INTO charactercreator_character
    (name, level, exp, hp, strength, intelligence, dexterity, wisdom)
    VALUES """ + str(character[1:]) + ';'
  pg_curs.execute(insert_character)

In [24]:
pg_curs.execute('SELECT * FROM charactercreator_character')
pg_curs.fetchall()

[(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1),
 (2, 'Optio dolorem ex a', 0, 0, 10, 1, 1, 1, 1),
 (3, 'Minus c', 0, 0, 10, 1, 1, 1, 1),
 (4, 'Sit ut repr', 0, 0, 10, 1, 1, 1, 1),
 (5, 'At id recusandae expl', 0, 0, 10, 1, 1, 1, 1),
 (6, 'Non nobis et of', 0, 0, 10, 1, 1, 1, 1),
 (7, 'Perferendis', 0, 0, 10, 1, 1, 1, 1),
 (8, 'Accusantium amet quidem eve', 0, 0, 10, 1, 1, 1, 1),
 (9, 'Sed nostrum inventore error m', 0, 0, 10, 1, 1, 1, 1),
 (10, 'Harum repellendus omnis od', 0, 0, 10, 1, 1, 1, 1),
 (11, 'Itaque ut commodi,', 0, 0, 10, 1, 1, 1, 1),
 (12, 'Molestiae quis', 0, 0, 10, 1, 1, 1, 1),
 (13, 'Ali', 0, 0, 10, 1, 1, 1, 1),
 (14, 'Tempora quod optio possimus il', 0, 0, 10, 1, 1, 1, 1),
 (15, 'Sed itaque beatae pari', 0, 0, 10, 1, 1, 1, 1),
 (16, 'Quam dolor', 0, 0, 10, 1, 1, 1, 1),
 (17, 'Molestias expedita', 0, 0, 10, 1, 1, 1, 1),
 (18, 'Lauda', 0, 0, 10, 1, 1, 1, 1),
 (19, 'Incidunt sint perferen', 0, 0, 10, 1, 1, 1, 1),
 (20, 'Laboriosa', 0, 0, 10, 1, 1, 1, 1),
 (21,

In [0]:
pg_curs.close()
pg_conn.commit()

# table is now committed to ElephantSQL

In [0]:
pg_curs = pg_conn.cursor()
pg_curs.execute('SELECT * FROM charactercreator_character')
pg_characters = pg_curs.fetchall()

In [0]:
for character,pg_character in zip(characters, pg_characters):
  assert pg_character == character

In [0]:
# Nothing happened - which means they're all the same!
# We have successfully performed and validated an ETL